# LLM Reflection

AutoGen supports different LLM prompting and reasoning strategies, such as ReAct, Reflection/Self-Critique, and more. This notebook demonstrates how to realize general LLM reflection with AutoGen. Reflection is a general prompting strategy which involves having LLMs analyze their own outputs, behaviors, knowledge, or reasoning processes.

This example leverages a generic interface [nested chats](/docs/tutorial/conversation-patterns#nested-chats) to implement the reflection using multi-agents.

In [5]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
)
llm_config = {
    "config_list": config_list,  # Assuming you have this defined elsewhere
    "timeout": 120,
}

## Construct Agents
Now we create three agents, including `user_proxy` as a user proxy, `writing_assistant` for generating solutions (based on the initial request or critique), and `reflection_assistant` for reflecting and providing critique.

In [6]:
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=10,
    code_execution_config={
        # "work_dir": "coding",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

writing_assistant = autogen.AssistantAgent(
    name="writing_assistant",
    system_message="You are an writing assistant tasked to write engaging blogpost. You try generate the best blogpost possible for the user's request. If the user provides critique, respond with a revised version of your previous attempts.",
    llm_config=llm_config,
)

reflection_assistant = autogen.AssistantAgent(
    name="reflection_assistant",
    system_message="Generate critique and recommendations on the writing. Provide detailed recommendations, including requests for length, depth, style, etc..",
    llm_config=llm_config,
)

## Construct Agent Chats with `reflection_assistant` being a Nested Agent for Reflection

In [7]:
def reflection_message(recipient, messages, sender, config):
    print("Reflecting...", "yellow")
    return f"Reflect and provide critique on the following writing. \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}"


nested_chat_queue = [
    {
        "recipient": reflection_assistant,
        "message": reflection_message,
        "max_turns": 1,
    },
]
user_proxy.register_nested_chats(
    nested_chat_queue,
    trigger=writing_assistant,
    # position=4,
)

user_proxy.initiate_chat(
    writing_assistant,
    message="Write an engaging blogpost on the recent updates in AI. The blogpost should be engaging and understandable for general audience. Should have more than 3 paragraphes but no longer than 1000 words.",
    max_turns=2,
)

user_proxy (to writing_assistant):

Write an engaging blogpost on the recent updates in AI. The blogpost should be engaging and understandable for general audience. Should have more than 3 paragraphes but no longer than 1000 words.

--------------------------------------------------------------------------------
writing_assistant (to user_proxy):

## Navigating the New Frontiers of Artificial Intelligence: A User-Friendly Update

Artificial Intelligence (AI) is leaping forward at an exhilarating speed, reshaping our world and how we interact with it. With a raft of recent updates, AI is becoming more intuitive, capable, and integrated into our daily lives. If you've ever felt overwhelmed by tech talk and jargon, worry not! Let's break down the latest in AI, making it not only digestible for everyone but also highlighting why it should matter to you.

### AI Gets Creative: GPT-4 and More!

Have you heard of ChatGPT? This AI, which made waves for mimicking human-like conversation, has no

ChatResult(chat_id=None, chat_history=[{'content': 'Write an engaging blogpost on the recent updates in AI. The blogpost should be engaging and understandable for general audience. Should have more than 3 paragraphes but no longer than 1000 words.', 'role': 'assistant'}, {'content': "## Navigating the New Frontiers of Artificial Intelligence: A User-Friendly Update\n\nArtificial Intelligence (AI) is leaping forward at an exhilarating speed, reshaping our world and how we interact with it. With a raft of recent updates, AI is becoming more intuitive, capable, and integrated into our daily lives. If you've ever felt overwhelmed by tech talk and jargon, worry not! Let's break down the latest in AI, making it not only digestible for everyone but also highlighting why it should matter to you.\n\n### AI Gets Creative: GPT-4 and More!\n\nHave you heard of ChatGPT? This AI, which made waves for mimicking human-like conversation, has now evolved into its fourth iteration, GPT-4. Imagine an AI t